In [2]:
import numpy as np

path = "/work/a06/tsuda/2025_master_Tsuda/sotsuron/energy_transition_analysis/new_macro/Englandwales.npy"    # ← まずこれなど1つ指定
data = np.load(path, allow_pickle=True)

print("type:", type(data))
print("dtype:", getattr(data, "dtype", None))

print("\n=== raw data ===")
print(data)

# もし dict っぽければ keys を見る
if isinstance(data, dict):
    print("\nkeys:", data.keys())

# もし object 配列なら item() をチェック
if data.__class__.__name__ == "ndarray" and data.dtype == object:
    try:
        print("\nitem():", data.item())
    except:
        pass


type: <class 'numpy.ndarray'>
dtype: object

=== raw data ===
{'Year': array([1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810,
       1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821,
       1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832,
       1833, 1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843,
       1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854,
       1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865,
       1866, 1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876,
       1877, 1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887,
       1888, 1889, 1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898,
       1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909,
       1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920,
       1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931,
       1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939

In [ ]:
import numpy as np
import pandas as pd
import os
import glob

# ==== 設定 ====
# 10か国の npy があるフォルダ
npy_dir = "."   # ノートブックと同じ場所なら "." のままでOK

# 対象年
target_years = [1850, 1950, 2000]

# 出力ファイル名
out_csv_long = "modern_energy_10countries_long.csv"
out_csv_wide = "modern_energy_10countries_wide.csv"

rows = []

# ==== 10か国ぶんの npy を読み込む ====
# 明示的に指定したいなら ↓ のように書き換えてもOK
# npy_files = [
#     "Englandwales.npy", "France.npy", "Germany.npy", "Italy.npy",
#     "Netherland.npy", "Portugal.npy", "Spain.npy", "Sweden.npy",
#     "Uruguay.npy", "Japan.npy",
# ]
# for fname in npy_files:
#     path = os.path.join(npy_dir, fname)

for path in glob.glob(os.path.join(npy_dir, "*.npy")):
    country = os.path.splitext(os.path.basename(path))[0]

    # npy 読み込み（dict が1つ入った object ndarray）
    raw = np.load(path, allow_pickle=True)
    data = raw.item()

    years = np.asarray(data["Year"])
    modern = np.asarray(data["modern_rate"])
    modern_norm = np.asarray(data["modern_rate_norm"])

    # ---- 各国について 1850/1950/2000 を抽出 ----
    for y in target_years:
        idx = np.where(years == y)[0]

        if len(idx) == 0:
            mr = np.nan
            mrn = np.nan
        else:
            i = idx[0]
            mr = float(modern[i])
            mrn = float(modern_norm[i])

        rows.append({
            "Country": country,
            "Year": int(y),
            "modern_rate": mr,
            "modern_rate_norm": mrn,
        })

# ==== 長い形式（Country × Year）====
df_long = pd.DataFrame(rows).sort_values(["Country", "Year"])
df_long.to_csv(out_csv_long, index=False, encoding="utf-8-sig")
print("Saved long-format CSV ->", out_csv_long)
display(df_long.head())

# ==== 横持ち形式（1行=1国）====
pivot_modern = df_long.pivot(index="Country", columns="Year", values="modern_rate")
pivot_norm = df_long.pivot(index="Country", columns="Year", values="modern_rate_norm")

pivot_modern.columns = [f"modern_{c}" for c in pivot_modern.columns]
pivot_norm.columns = [f"norm_{c}" for c in pivot_norm.columns]

df_wide = pd.concat([pivot_modern, pivot_norm], axis=1).reset_index()
df_wide.to_csv(out_csv_wide, index=False, encoding="utf-8-sig")
print("Saved wide-format CSV ->", out_csv_wide)
display(df_wide.head())


In [3]:
import numpy as np
import pandas as pd
import os

# ===== 設定 =====
npy_dir = "/work/a06/tsuda/2025_master_Tsuda/sotsuron/energy_transition_analysis/new_macro"

# 10か国
npy_files = [
    "Canada.npy",
    "Englandwales.npy",
    "France.npy",
    "Germany.npy",
    "Italy.npy",
    "Netherland.npy",
    "Portugal.npy",
    "Spain.npy",
    "Sweden.npy",
    "Uruguay.npy",
]

target_years = [1850, 1950, 2000]

out_csv = os.path.join(npy_dir, "modern_energy_10countries.csv")

rows = []

# ===== 各国 npy を読み込む =====
for fname in npy_files:
    path = os.path.join(npy_dir, fname)
    country = os.path.splitext(fname)[0]

    raw = np.load(path, allow_pickle=True)
    data = raw.item()  # dict 取り出し

    years = np.asarray(data["Year"])
    modern = np.asarray(data["modern_rate"])
    modern_norm = np.asarray(data["modern_rate_norm"])

    row = {"Country": country}

    # ---- ターゲット年を抜き取る ----
    for y in target_years:
        idx = np.where(years == y)[0]

        if len(idx) == 0:
            row[f"modern_{y}"] = np.nan
            row[f"norm_{y}"] = np.nan
        else:
            i = idx[0]
            row[f"modern_{y}"] = float(modern[i])
            row[f"norm_{y}"] = float(modern_norm[i])

    rows.append(row)

# ===== DataFrame 化 → CSV 出力 =====
df = pd.DataFrame(rows).sort_values("Country")
df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print("Saved:", out_csv)
display(df)


Saved: /work/a06/tsuda/2025_master_Tsuda/sotsuron/energy_transition_analysis/new_macro/modern_energy_10countries.csv


,Country,modern_1850,norm_1850,modern_1950,norm_1950,modern_2000,norm_2000
0,Canada,0.034,0.034588,0.838,0.852492,0.980,0.996948
1,Englandwales,0.917,0.934760,0.973,0.991845,0.971,0.989806
2,France,0.270,0.294438,0.811,0.884406,0.897,0.978190
3,Germany,0.276,0.285419,0.923,0.954498,0.946,0.978283
4,Italy,NaN,NaN,0.554,0.592513,0.931,0.995722
5,Netherland,0.186,0.190769,0.901,0.924103,0.910,0.933333
6,Portugal,NaN,NaN,0.311,0.372455,0.832,0.996407
7,Spain,0.016,0.017335,0.614,0.665222,0.916,0.992416
8,Sweden,0.020,0.022321,0.732,0.816964,0.729,0.813616
9,Uruguay,NaN,NaN,0.686,0.821557,0.835,1.000000
